#This code will compare two files for Pfam domains

In [1]:
#!/usr/bin/python
from collections import defaultdict
import pandas as pd
from os.path import exists, join
import pdb
import os
import re
import csv

In [14]:
input_hmmer_dir = "../output/hmmer_txt_output/braker3_gff"
input_dir = "../input/"

output_dir = "../output/hmmer_txt_output/braker3_gff/TLR_output_files"
#output_file_pfam = "m_capitata_2_Pfam_domain_similarity_braker3_gff_lrr_tir.tsv"
output_filepath = join(output_dir,output_file_pfam)


required = [input_dir,input_hmmer_dir,output_dir]

for r in required:
    if not exists(r):
        raise FileNotFoundError(f"The specified hmmer file {r} does not seem to be in the specified path.")
    print(f"Confirmed that required file or directory {r} exists.")


Confirmed that required file or directory ../input/ exists.
Confirmed that required file or directory ../output/hmmer_txt_output/braker3_gff exists.
Confirmed that required file or directory ../output/hmmer_txt_output/braker3_gff/TLR_output_files exists.


## Domains to choose for IG or LLR PFam domains
domains_for Ig: PF00047,PF18452,PF13927,PF01582,PF13895 (To build a IL-1R)

domains for lrrs: PF00560, PF07723, PF07725, PF12799, PF13306, PF13516, PF13855, PF14580, PF18805, PF18837 (to build a TLR)

In [15]:

def get_hmmer_results_by_species (coral_species, input_hmmer_dir):
    """ Return a dict mapping domain ids to file paths for hmmer results.
    coral_species -- a string of species (code assumes file starts with species name)
    input_hmmer_dir -- a string representing the path to a directory holding the hmmer results file for each domain 
    """
    hmmer_results_by_domain = {}
    for file in os.listdir(input_hmmer_dir):
        if not file.startswith(coral_species):
            continue
        if not file.endswith('hmmer_braker3_gff_results.txt'):
        #if not file.endswith('profile_hmmer_results.txt'):
            continue
            #pfam_domain = os.path.basename(file("_")[-1])#change to -1 for nontest sequences
        pf_domain = file.split("_proteins_")[1]
        pf_domain = pf_domain.split("_")[0]
        hmmer_results_by_domain[pf_domain] = join(input_hmmer_dir,file)
    return hmmer_results_by_domain


def gene_ids_with_motif(coral_species, motif,domains_by_motif, hmmer_results_by_domain):
    #open the folder
    gene_ids_with_motif = set()
    result_df = None
    #pfam_domains = set()
    for domain in domains_by_motif[motif]:
        print(f"Finding genes with domain {domain}")
        domains = domain
        #open the file for that domain
        hmmer_results_file = hmmer_results_by_domain[domain]
        print(f"Parsing hmmer results file {hmmer_results_file}")
        df = pd.read_csv(hmmer_results_file,delim_whitespace=True, comment="#")
        #print(repr(df.columns))
        df.columns=["target_name", "accession", "query_name", "accession", "E-value_full_seq", "score_full_seq", "bias_full_seq", 
        "E-value", "score", "bias", "exp", "reg", "clu", "ov", "env", "dom", "rep", "inc", "description_of_target"]
        #df_filtered = df[df["E-value_full_seq"]<0.0000000001]
        df_filtered = df[df["E-value_full_seq"]<0.01]
        df_filtered["motif"] = motif
        df_filtered["species"] = coral_species
        if result_df is None:
            result_df = df_filtered
        else:
            result_df = pd.concat([df_filtered, result_df], ignore_index = True, axis = 0)
        gene_ids = result_df["target_name"]
        pfam_domains = result_df["query_name"]
        pfam_domains = list(set(list(pfam_domains)))
        pfam_domains = sorted(pfam_domains)
        #print(gene_ids)
        gene_ids_stripped = gene_ids.str.split('.t').str[0] 
        #gene_ids_stripped = gene_ids_stripped.str.split('.').str[-1]
        #print(gene_ids_strip)
        gene_id_union = gene_ids_with_motif.union(set(gene_ids_stripped))
        gene_ids_with_motif = gene_id_union
        #pfam_domains = df_filter["query_name"]
        #print(domains)
    return gene_ids_with_motif, result_df
    #result_pfam = gene_ids_with_motif()
    #print(result_pfam)

def identify_gene_family_by_motif(coral_species, required_motifs, domains_by_motif, hmmer_results_by_domain):
        df = None
        for i,motif in enumerate(required_motifs):
            print("the current motif is", motif)
            current_gene_ids, current_df = gene_ids_with_motif(coral_species, motif,domains_by_motif, hmmer_results_by_domain)
            print("the following genes have this motif", current_gene_ids)
            if df is None:
                df = current_df
            else:
                df = pd.concat([df, current_df], ignore_index = True, axis = 0)

            #For the first motif, whichever genes 
            #have it are our matching genes to 
            #start out with
            if i == 0:
                matching_genes = set(current_gene_ids)
                continue

            matching_genes = matching_genes.intersection(current_gene_ids)
        print(f"Gene with motifs {required_motifs}: {matching_genes}")
        #now filter the df with all genes to just those with all the required motifes
        print("example target names:", list(df["target_name"]))
        df = df[df["target_name"].str.split('.t').str[0].isin(matching_genes)]

        return list(matching_genes),df


motifs_by_gene_family = {"TLR":["LRR","TIR"]}
domains_by_motif = {"LRR":["PF00560", "PF07723", "PF12799", "PF13306", "PF13516", "PF13855", "PF14580"],"TIR":["PF01582"]}
#PF07725

coral_species = 'm_capitata'
hmmer_results_by_domain = get_hmmer_results_by_species(coral_species, input_hmmer_dir)
gene_ids_by_gene_family = {}
df = None
for gene_family in motifs_by_gene_family.keys():
    print(f"Finding genes that are in the {gene_family} gene family")
    #gene_ids_with_motif
    required_motifs = motifs_by_gene_family[gene_family]
    print(f"{gene_family} is defined by the following motifs: {required_motifs}")
    gene_ids, current_df = identify_gene_family_by_motif(coral_species, required_motifs, domains_by_motif, hmmer_results_by_domain)
    current_df["gene_family"] = gene_family
    if df is None:
        df = current_df
    else:
        df = pd.concat([df, current_df], ignore_index = True, axis = 0)
    gene_ids_by_gene_family[gene_family] = gene_ids
print("gene ids by gene family", gene_ids_by_gene_family)
gene_id_split = df["target_name"].str.split('.t').str[0]
gene_id_split_end = gene_id_split.str.split('.').str[-1]
df["gene_id"]=gene_id_split_end
print(df)
df.to_csv(output_filepath)
 

Finding genes that are in the TLR gene family
TLR is defined by the following motifs: ['LRR', 'TIR']
the current motif is LRR
Finding genes with domain PF00560
Parsing hmmer results file ../output/hmmer_txt_output/braker3_gff/m_capitata_soft_mask_braker_filtered_proteins_PF00560_full_profile_hmmer_braker3_gff_results.txt
Finding genes with domain PF07723
Parsing hmmer results file ../output/hmmer_txt_output/braker3_gff/m_capitata_soft_mask_braker_filtered_proteins_PF07723_full_profile_hmmer_braker3_gff_results.txt
Finding genes with domain PF12799
Parsing hmmer results file ../output/hmmer_txt_output/braker3_gff/m_capitata_soft_mask_braker_filtered_proteins_PF12799_full_profile_hmmer_braker3_gff_results.txt
Finding genes with domain PF13306
Parsing hmmer results file ../output/hmmer_txt_output/braker3_gff/m_capitata_soft_mask_braker_filtered_proteins_PF13306_full_profile_hmmer_braker3_gff_results.txt
Finding genes with domain PF13516
Parsing hmmer results file ../output/hmmer_txt_outpu

/tmp/ipykernel_5555/2016752920.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["motif"] = motif
/tmp/ipykernel_5555/2016752920.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["species"] = coral_species
/tmp/ipykernel_5555/2016752920.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [7]:
#I don'think we need this
#Need to first define the motifs we are searching for
motifs_by_gene_family = {"TLR":["LRR","TIR"],"IL-1R":["Ig","TIR"]}
domains_by_motif = {"LRR":["PF00560","PF07723","PF07725"],"TIR":["PF01582"]}

for gene_family in motifs_by_gene_family.keys():
    print(f"Finding genes that are in the {gene_family} gene family")

Finding genes that are in the TLR gene family
Finding genes that are in the IL-1R gene family


In [7]:

#domains_by_gene_family = {"TLR":["PF00047","PF18452","PF13927","PF01582","PF13895"]}
#domains for lrrs: PF00560, PF07723, PF07725, PF12799, PF13306, PF13516, PF13855, PF14580, PF18805, PF18837
domains_by_gene_family = {"TLR":["PF18452","PF18805"]}
filename_template="a_cytherea_soft_mask_braker_{domain}_full_profile_hmmer_braker3_results.txt"
for gene_family in domains_by_gene_family.keys():
    domains=domains_by_gene_family[gene_family]
    print(domains)
    results ={}
    for domain in domains:
        #figure out file name for current domain
        domain_file=filename_template.format(domain=domain)
        print(domain_file)
        domain_file_path=os.path.join(input_hmmer_dir,domain_file)
        print(domain_file_path)
        for domain_2 in domains:
            domain_file_2=filename_template.format(domain=domain_2)
            print(domain_file_2)
            domain_file_path_2=os.path.join(input_hmmer_dir,domain_file_2)
            print(domain_file_path_2)
            gene_ids=find_shared_domains(domain_file_path,domain_file_path_2)
            print(gene_ids)
            if not results:
                results=gene_ids
            results=results.intersection(gene_ids)
            print(len(results))
            

['PF18452', 'PF18805']
a_cytherea_soft_mask_braker_PF18452_full_profile_hmmer_braker3_results.txt
../output/hmmer_txt_output/braker3_gff/a_cytherea_soft_mask_braker_PF18452_full_profile_hmmer_braker3_results.txt
a_cytherea_soft_mask_braker_PF18452_full_profile_hmmer_braker3_results.txt
../output/hmmer_txt_output/braker3_gff/a_cytherea_soft_mask_braker_PF18452_full_profile_hmmer_braker3_results.txt


NameError: name 'find_shared_domains' is not defined

In [10]:
#Need to extract some parts of the files used to write to the common output file.
species = os.path.basename(domain_file[:domain_file.index('_soft')])
print(species)
#pfam_file_1 = os.path.basename(domain_file.split("_")[-2])#change to -1 for nontest sequences
#print(pfam_file_1)
#pfam_file_2 = os.path.basename(domain_file_2.split("_")[-2])#change to -1 for nontest sequences
#print(pfam_file_2)
pfam_string = str(domains)[1:-1]
pfam_replace_1 = pfam_string.replace("'","")
pfam_compare = pfam_replace_1.replace(",","_")
#pfam_compare = pfam_file_1 + " " + pfam_file_2
print(pfam_compare)


NameError: name 'domain_file' is not defined

In [36]:
print(gene_ids_by_gene_family)
print(gene_ids_by_gene_family['TLR'])

{'TLR': ['g11295', 'g21206', 'g3982', 'g18447', 'g6932', 'g9778']}
['g11295', 'g21206', 'g3982', 'g18447', 'g6932', 'g9778']


In [37]:
gene_id_intersection = gene_ids_by_gene_family['TLR']

In [25]:
#need to write each shared domain to a file
#NOTE: union comand releases output as a set. 

gene_ids_by_gene_family

with open(output_filepath, "a") as comparison_file:
    #writer=csv.DictWriter(comparison_file, fieldnames=shared_domains)
    for gene_id in results:
        print(f"Writing output file: {species}, {pfam_compare}, {gene_id}")
        #we have a problem in the output... it is seperating pfam compare into its own columns.
        #I want them as one column with all domains.
        row=[(species),(pfam_compare),(gene_id)]
        row="\t".join(map(str,row))
        comparison_file.write(row + "\n")

In [58]:
with open(output_filepath, "a") as comparison_file:
    #writer=csv.DictWriter(comparison_file, fieldnames=shared_domains)
    for gene_family in gene_id_intersection:
        print(f"Writing output file: {coral_species}, {gene_family}")
        #we have a problem in the output... it is seperating pfam compare into its own columns.
        #I want them as one column with all domains.
        #I still can't get the pfamdomain here which I think is important
        row=[(coral_species),(gene_family)]
        row="\t".join(map(str,row))
        comparison_file.write(row + "\n")

Writing output file: a_cytherea, g11295
Writing output file: a_cytherea, g21206
Writing output file: a_cytherea, g3982
Writing output file: a_cytherea, g18447
Writing output file: a_cytherea, g6932
Writing output file: a_cytherea, g9778


In [27]:
def greet(name,greeting="Hello"):
    """Print a greeting based on a name.
    name -- string of the name of the person to greet
    greeting -- string for the greeting
    """
    print(greeting, name)
help(greet)

Help on function greet in module __main__:

greet(name, greeting='Hello')
    Print a greeting based on a name.
    name -- string of the name of the person to greet
    greeting -- string for the greeting



In [18]:
#first loop through files in the output for hmmer output txt
for results_file in os.listdir(input_hmmer_dir_cnid):
    if not results_file.endswith(".txt"):
        continue
    #compare file names with the same species name
    species_name = results_file[:results_file.index('longest')]
    if species_name == species_name_2:
        filecmp.cmp(file1,file2)

NameError: name 'input_hmmer_dir_cnid' is not defined